# Connecting SIS and LWE to Lattices

So you're wondering why everyone says SIS and LWE are "lattice problems"? Let's figure that out with code and math!

**Credits to:**  
Alfred Menezes - [Lattice-Based Cryptography Course](https://www.youtube.com/playlist?list=PLA1qgQLL41STNFDvPJRqrHtuz0PIEJ4a8)

---

**What we'll explore:**
- Why SIS is really just finding short vectors in a lattice
- Why LWE is basically the "bounded distance decoding" problem
- How both problems connect to classical lattice problems like SVP

## The SIS Lattice - What's the Deal?

Remember the SIS problem? You get a random matrix $A \in \mathbb{Z}_q^{n \times m}$ and need to find a short vector $z$ such that $Az = 0 \pmod{q}$.

Turns out all solutions to $Az = 0 \pmod{q}$ form a lattice! We call it:

$$
L_A^\perp = \{z \in \mathbb{Z}^m : Az = 0 \pmod{q}\}
$$

This is literally just the kernel of $A$ modulo $q$. Every vector in this lattice satisfies $Az = 0 \pmod{q}$.

**Quick fact:** This is a lattice because it's a discrete additive subgroup of $\mathbb{R}^m$. You can add two solutions and get another solution, multiply by -1, all that good stuff.

In [5]:
import numpy as np

# Let's see L_A^perp in action with a tiny example
n, m, q = 2, 4, 7

A = np.array([[1, 2, 3, 1],
              [2, 1, 1, 3]]) % q

print(f"Matrix A ({n}×{m}) mod {q}:")
print(A)

# Find some vectors in the SIS lattice by brute force
print(f"\nVectors in L_A^perp (checking small ones):")
sis_vectors = []
for z1 in range(-5, 6):
    for z2 in range(-5, 6):
        for z3 in range(-5, 6):
            for z4 in range(-5, 6):
                z = np.array([z1, z2, z3, z4])
                if np.all((A @ z) % q == 0) and np.any(z != 0):
                    sis_vectors.append(z)
                    if len(sis_vectors) <= 5:  # Print first 5
                        print(f"z = {z}, ||z|| = {np.linalg.norm(z):.2f}")

print(f"\nFound {len(sis_vectors)} nonzero SIS solutions with small coordinates")

Matrix A (2×4) mod 7:
[[1 2 3 1]
 [2 1 1 3]]

Vectors in L_A^perp (checking small ones):
z = [-5 -5 -5 -5], ||z|| = 10.00
z = [-5 -5 -5  2], ||z|| = 8.89
z = [-5 -5  2 -5], ||z|| = 8.89
z = [-5 -5  2  2], ||z|| = 7.62
z = [-5 -4 -3  1], ||z|| = 7.14

Found 298 nonzero SIS solutions with small coordinates


## Rank and Structure of $L_A^\perp$

The SIS lattice has **full rank** $m$. Why? Because $q\mathbb{Z}^m$ sits inside it (all multiples of $q$ times any integer vector work). 

Since $q\mathbb{Z}^m$ already spans the whole space $\mathbb{R}^m$, our SIS lattice must too.

Also, $L_A^\perp$ is what's called a **$q$-ary lattice** - if you take any vector modulo $q$, it's still in the lattice.

In [6]:
# Check the q-ary property: z in lattice => z mod q also in lattice
q = 11
n, m = 2, 4

A = np.array([[3, 5, 2, 1],
              [1, 4, 3, 2]])

# Take some random vector in L_A^perp
z1 = np.array([7, -2, 1, 0])  # Let's check if this is in L_A^perp
if np.all((A @ z1) % q == 0):
    print(f"z1 = {z1} is in L_A^perp ✓")
    print(f"Az1 mod {q} = {(A @ z1) % q}")
    
    # Now add a multiple of q to some coordinates
    z2 = z1 + np.array([q, 0, -2*q, q])
    print(f"\nz2 = z1 + multiples of q = {z2}")
    print(f"Az2 mod {q} = {(A @ z2) % q}")
    
    if np.all((A @ z2) % q == 0):
        print("z2 is also in L_A^perp ✓ (q-ary property!)")
else:
    print("z1 not in lattice, trying another...")

z1 not in lattice, trying another...


## Volume of the SIS Lattice

Here's something cool: the volume (or determinant) of $L_A^\perp$ is exactly $q^n$.

Why? It's the dual lattice to $L_A = \{Az \bmod q : z \in \mathbb{Z}^m\}$, and there's a nice formula relating their volumes:

$$
\text{vol}(L_A^\perp) = \frac{1}{\text{vol}(L_A)} = q^n
$$

The bigger $q$ gets, the more "spread out" the SIS lattice becomes.

In [7]:
# Let's compute the volume for a simple example
n, m, q = 2, 3, 5

# We need A to have full rank n mod q
A = np.array([[1, 2, 1],
              [2, 1, 3]]) % q

print(f"Matrix A ({n}×{m}) mod {q}:")
print(A)

# Check rank
A_check = A[:, :n]
rank = np.linalg.matrix_rank(A_check % q)
print(f"\nRank check: {rank} (should be {n})")

# Volume prediction
expected_vol = q**n
print(f"\nExpected volume of L_A^perp: q^n = {q}^{n} = {expected_vol}")

# We can verify by constructing a basis (simplified approach)
# The actual volume depends on the basis we choose
print(f"\nThis means lattice points are spaced roughly {expected_vol**(1/m):.2f} apart per dimension")

Matrix A (2×3) mod 5:
[[1 2 1]
 [2 1 3]]

Rank check: 2 (should be 2)

Expected volume of L_A^perp: q^n = 5^2 = 25

This means lattice points are spaced roughly 2.92 apart per dimension


## Building a Basis for $L_A^\perp$

Want an actual basis for the SIS lattice? Here's a construction that works:

Split $A = [A_1 \ | \ A_2]$ where $A_2$ is invertible mod $q$. Then build:

$$
C = \begin{bmatrix} \tilde{A}_1 & I_n \\ qI_{m-n} & 0 \end{bmatrix}
$$

where $\tilde{A}_1 = -A_2^{-1}A_1 \bmod q$.

The first $n$ columns satisfy $Az = 0 \pmod{q}$, and the remaining columns are multiples of $q$ (automatically in the lattice).

In [8]:
# Building a basis for the SIS lattice
n, m, q = 2, 4, 7

A = np.array([[3, 1, 2, 5],
              [1, 2, 4, 3]]) % q

print(f"Matrix A ({n}×{m}) mod {q}:")
print(A)

# Split into A1 and A2 (last n columns for A2)
A1 = A[:, :m-n]
A2 = A[:, m-n:]

print(f"\nA1 (first {m-n} cols): \n{A1}")
print(f"\nA2 (last {n} cols): \n{A2}")

# Check if A2 is invertible mod q
det_A2 = int(round(np.linalg.det(A2))) % q
print(f"\ndet(A2) mod {q} = {det_A2}")

if det_A2 != 0:
    print("A2 is invertible! ✓")
    
    # Build the basis matrix (simplified version)
    C_topright = np.eye(n)
    C_topleft = A1
    C_bottomleft = q * np.eye(m-n)
    C_bottomright = np.zeros((m-n, n))
    
    C = np.block([[C_topleft, C_topright],
                  [C_bottomleft, C_bottomright]])
    
    print(f"\nBasis matrix C:")
    print(C.astype(int))
    
    # Verify columns are in L_A^perp
    print(f"\nVerifying first column is in L_A^perp:")
    col0 = C[:, 0]
    result = (A @ col0) % q
    print(f"A × col_0 mod {q} = {result}")
    print("Should be [0, 0] ✓" if np.all(result == 0) else "Oops!")

Matrix A (2×4) mod 7:
[[3 1 2 5]
 [1 2 4 3]]

A1 (first 2 cols): 
[[3 1]
 [1 2]]

A2 (last 2 cols): 
[[2 5]
 [4 3]]

det(A2) mod 7 = 0


## How to Solve SIS = Solving SVP

Now the big reveal: **solving SIS is just finding short vectors in $L_A^\perp$!**

If you can solve approximate-SVP (find a vector with norm $\leq \gamma \cdot \lambda_1$), you can solve SIS as long as:

$$
\beta \geq \gamma \cdot \lambda_1(L_A^\perp)
$$

Using the Gaussian heuristic, we expect $\lambda_1 \approx \sqrt{m/n} \cdot q^{n/m}$. So if your bound $\beta$ is bigger than this, there should be short solutions!

**SIS2 variant:** If you have a syndrome $y$ (need $Az = y \pmod{q}$), this becomes the Closest Vector Problem (CVP) or Bounded Distance Decoding (BDD) instead.

In [9]:
# Estimating lambda_1 with Gaussian heuristic
n, m, q = 10, 30, 1009

# Gaussian heuristic for lambda_1
lambda_1_estimate = np.sqrt(m / (2 * np.pi * np.e)) * (q ** (n/m))

print(f"Parameters: n={n}, m={m}, q={q}")
print(f"\nGaussian heuristic estimate:")
print(f"λ₁(L_A^perp) ≈ √(m/(2πe)) × q^(n/m)")
print(f"            ≈ {lambda_1_estimate:.2f}")

# What beta do we need for SIS to be solvable?
gamma = 1.5  # approximation factor
beta_needed = gamma * lambda_1_estimate

print(f"\nFor approximate-SVP with γ={gamma}:")
print(f"Need β ≥ {beta_needed:.2f}")

# Volume of the lattice
vol = q ** n
print(f"\nLattice volume: q^n = {vol:.2e}")
print(f"Average spacing: vol^(1/m) = {vol**(1/m):.2f}")

Parameters: n=10, m=30, q=1009

Gaussian heuristic estimate:
λ₁(L_A^perp) ≈ √(m/(2πe)) × q^(n/m)
            ≈ 13.29

For approximate-SVP with γ=1.5:
Need β ≥ 19.94

Lattice volume: q^n = 1.09e+30
Average spacing: vol^(1/m) = 10.03


## Why SIS is Actually Hard (Average-Case)

Here's the kicker: SIS isn't just hard, it's **provably hard** based on worst-case lattice problems!

**Ajtai (1996):** If you can solve SIS for random $A$, you can solve $\text{SIVP}_\gamma$ for ANY lattice (worst-case).

**Micciancio-Regev (2007):** Improved it to show solving SIS ⟹ solving worst-case $\text{SVP}_\gamma$ with $\gamma = O(\sqrt{n})$.

This means even if you only care about random instances of SIS, you'd need to break the hardest possible lattice problems. That's the magic of worst-case to average-case reductions!

In [10]:
# Visualizing the hardness: approximation factors needed
n_values = np.array([100, 200, 300, 400, 500])

# Micciancio-Regev: gamma = O(sqrt(n))
gamma_MR = np.sqrt(n_values)

print("Security based on SVP approximation factors:")
print("=" * 50)
print(f"{'n':<10} {'γ (approx.)':<15} {'Hardness'}")
print("-" * 50)

for i, n in enumerate(n_values):
    gamma = gamma_MR[i]
    # Rough security estimate (very simplified)
    bits = n / 2  # Very rough estimate
    print(f"{n:<10} {gamma:<15.1f} ~{bits:.0f}-bit")

print("\nThis is why lattice crypto is post-quantum secure!")
print("Even quantum computers struggle with SVP approximation.")

Security based on SVP approximation factors:
n          γ (approx.)     Hardness
--------------------------------------------------
100        10.0            ~50-bit
200        14.1            ~100-bit
300        17.3            ~150-bit
400        20.0            ~200-bit
500        22.4            ~250-bit

This is why lattice crypto is post-quantum secure!
Even quantum computers struggle with SVP approximation.


## The LWE Lattice - A Different Flavor

Now for LWE! Remember, you get $(A, b)$ where $b = As + e \pmod{q}$ with small error $e$, and need to find $s$.

The lattice here is:

$$
L_A = \{Az \bmod q : z \in \mathbb{Z}^n\} \subseteq \mathbb{Z}^m
$$

This is the column span of $A$ modulo $q$. The vector $b$ is **close** to a lattice point ($As$), separated only by the small error $e$.

So LWE becomes: given $b$ near the lattice, find the closest lattice point $As$. That's the **Bounded Distance Decoding (BDD)** problem!

In [11]:
# LWE as a BDD problem
n, m, q = 3, 5, 13

A = np.array([[2, 1, 4, 3, 5],
              [1, 3, 2, 5, 4],
              [3, 2, 1, 4, 2]]) % q

s = np.array([3, 5, 2])
e = np.array([1, -1, 0, 1, -1])  # Small error

# Compute b = As + e
As = (A.T @ s) % q
b = (As + e) % q

print(f"Secret s = {s}")
print(f"Error e = {e}")
print(f"\nAs mod {q} = {As}")
print(f"b = As + e mod {q} = {b}")

# The lattice L_A consists of all Az mod q
print(f"\nThe lattice L_A = {{Az mod {q} : z ∈ Z^{n}}}") 
print(f"As is in this lattice, and b is close to As")
print(f"\nDistance: ||b - As|| = ||e|| = {np.linalg.norm(e):.2f}")

# Generate a few lattice points to visualize
print(f"\nSome other lattice points:")
for z in [[1,0,0], [0,1,0], [0,0,1], [1,1,1]]:
    z_arr = np.array(z)
    point = (A.T @ z_arr) % q
    dist = np.linalg.norm(b - point)
    print(f"z={z}: Az mod {q} = {point}, distance to b = {dist:.2f}")

Secret s = [3 5 2]
Error e = [ 1 -1  0  1 -1]

As mod 13 = [ 4  9 11  3  0]
b = As + e mod 13 = [ 5  8 11  4 12]

The lattice L_A = {Az mod 13 : z ∈ Z^3}
As is in this lattice, and b is close to As

Distance: ||b - As|| = ||e|| = 2.00

Some other lattice points:
z=[1, 0, 0]: Az mod 13 = [2 1 4 3 5], distance to b = 12.53
z=[0, 1, 0]: Az mod 13 = [1 3 2 5 4], distance to b = 13.67
z=[0, 0, 1]: Az mod 13 = [3 2 1 4 2], distance to b = 15.49
z=[1, 1, 1]: Az mod 13 = [ 6  6  7 12 11], distance to b = 9.27


## Structure of the LWE Lattice

$L_A$ has dimension $n$ (not full rank like the SIS lattice). It's embedded in $\mathbb{R}^m$ but only spans an $n$-dimensional subspace.

The volume is roughly $q^{m-n}$, which makes sense - there are $q^m$ possible outputs but they collapse to $q^n$ equivalence classes.

In [ ]:
# Understanding the dimension of L_A
n, m, q = 3, 6, 11

print(f"Parameters: n={n}, m={m}, q={q}")
print(f"\nL_A has dimension {n} (not full rank!)")
print(f"It's a {n}-dimensional lattice living in {m}-dimensional space")

# Volume estimate
vol_estimate = q ** (m - n)
print(f"\nVolume estimate: q^(m-n) = {q}^{m-n} = {vol_estimate}")

# Density of lattice points
density = vol_estimate ** (1/n)
print(f"Average spacing between lattice points: {density:.2f}")

# Compare to error size
typical_error_norm = np.sqrt(m)  # Assuming errors from {-1,0,1}
print(f"\nTypical error ||e||: ~{typical_error_norm:.2f}")
print(f"For LWE to be secure: error < λ₁(L_A)/2")
print(f"Otherwise decoding becomes easy!")

## Solving LWE = Solving BDD

To solve LWE, we need to find the closest lattice point in $L_A$ to the target $b$. That's exactly the BDD problem!

**Bounded Distance Decoding:** Given a lattice $L$ and a target $t$ that's close to some lattice point (distance $< \lambda_1/2$), find that lattice point.

For LWE:
- Lattice = $L_A$
- Target = $b$
- Close point = $As$ (with distance $\|e\|$)

If $\|e\| < \lambda_1(L_A)/2$, BDD will uniquely recover $As$, and we can then find $s$!

In [12]:
# Simulating BDD for LWE
n, m, q = 2, 4, 7

A = np.array([[1, 2, 3, 4],
              [2, 3, 1, 2]]) % q

s = np.array([3, 2])
e = np.array([1, 0, -1, 1])  # Small error

As = (A.T @ s) % q
b = (As + e) % q

print("LWE Instance:")
print(f"b = {b}")
print(f"Secret s = {s} (unknown to attacker)")
print(f"\nGoal: Find As in lattice L_A closest to b")

# Brute force search for small z (simulate BDD)
print(f"\nSearching lattice points Az for z with small coordinates:")
best_z = None
best_dist = float('inf')

for z1 in range(-3, 4):
    for z2 in range(-3, 4):
        z = np.array([z1, z2])
        Az = (A.T @ z) % q
        # Distance in "wrapped" space is tricky, use simple Euclidean
        dist = np.linalg.norm(b - Az)
        if dist < best_dist:
            best_dist = dist
            best_z = z
            
print(f"\nClosest lattice point found:")
print(f"z = {best_z} (should be {s}!)")
print(f"Az mod {q} = {(A.T @ best_z) % q}")
print(f"Distance = {best_dist:.2f}")

if np.array_equal(best_z % q, s % q):
    print("\n✓ Successfully recovered s!")

LWE Instance:
b = [1 5 3 3]
Secret s = [3 2] (unknown to attacker)

Goal: Find As in lattice L_A closest to b

Searching lattice points Az for z with small coordinates:

Closest lattice point found:
z = [3 2] (should be [3 2]!)
Az mod 7 = [0 5 4 2]
Distance = 1.73

✓ Successfully recovered s!


## The Magic Trick: BDD → SVP (Kannan's Embedding)

You can reduce BDD to SVP using a clever embedding!

Given BDD instance (lattice $L$, target $t$), build a new $(m+1)$-dimensional lattice:

$$
L' = \text{lattice with basis } \begin{bmatrix} B & t \\ 0 & M \end{bmatrix}
$$

where $B$ is a basis for $L$ and $M$ is a scaling parameter.

**The trick:** If $v \in L$ is the closest point to $t$, then the shortest vector in $L'$ is:

$$
\begin{bmatrix} t - v \\ M \end{bmatrix} = \begin{bmatrix} e \\ M \end{bmatrix}
$$

So finding the shortest vector in $L'$ gives you the error $e = t - v$, which reveals $v$!

In [13]:
# Demonstrating Kannan's embedding
print("Kannan's Embedding: BDD → SVP")
print("=" * 50)

# Simple 2D example
B = np.array([[3, 0],
              [1, 2]], dtype=float)  # Basis for lattice L

t = np.array([5.5, 3.2])  # Target (close to lattice)
v = np.array([6, 4])  # Closest lattice point (2*col1 + 2*col2)
e = t - v  # Error vector

M = 1  # Scaling parameter

print(f"Original lattice basis B:")
print(B)
print(f"\nTarget t = {t}")
print(f"Closest point v = {v}")
print(f"Error e = t - v = {e}")

# Build embedded lattice L'
B_extended = np.column_stack([B, t])
B_extended = np.vstack([B_extended, [0, 0, M]])

print(f"\nEmbedded lattice basis B':")
print(B_extended)

# The shortest vector should be [e1, e2, M]
shortest = np.array([e[0], e[1], M])
print(f"\nExpected shortest vector in L': {shortest}")
print(f"Norm: {np.linalg.norm(shortest):.3f}")

# Any lattice vector with last coord = 0 has norm ≥ λ₁(L)
lambda_1_original = np.min([np.linalg.norm(B[:, i]) for i in range(2)])
print(f"\nλ₁(original lattice) ≈ {lambda_1_original:.3f}")
print(f"Norm of shortest vector in L' = {np.linalg.norm(shortest):.3f}")
print(f"\n✓ If M chosen right, this is the shortest vector!")

Kannan's Embedding: BDD → SVP
Original lattice basis B:
[[3. 0.]
 [1. 2.]]

Target t = [5.5 3.2]
Closest point v = [6 4]
Error e = t - v = [-0.5 -0.8]

Embedded lattice basis B':
[[3.  0.  5.5]
 [1.  2.  3.2]
 [0.  0.  1. ]]

Expected shortest vector in L': [-0.5 -0.8  1. ]
Norm: 1.375

λ₁(original lattice) ≈ 2.000
Norm of shortest vector in L' = 1.375

✓ If M chosen right, this is the shortest vector!


## Why LWE is Hard (Regev's Reduction)

Just like SIS, LWE has a worst-case to average-case reduction!

**Regev (2005):** Solving LWE on average (for random $A$) is as hard as solving worst-case $\text{GapSVP}_\gamma$ and $\text{SIVP}_\gamma$ with $\gamma = \tilde{O}(n/\alpha)$.

The catch? Regev's original reduction uses **quantum** algorithms. But later work (Peikert, Brakerski et al.) found **classical** reductions under certain conditions.

Bottom line: breaking LWE means breaking the hardest lattice problems out there.

In [ ]:
# Regev's parameters and approximation factors
n_values = np.array([128, 192, 256, 384, 512])
q_values = np.array([2048, 4096, 8192, 16384, 32768])
alpha_values = np.array([0.01, 0.008, 0.006, 0.005, 0.004])

print("Regev's Reduction: LWE Parameters")
print("=" * 60)
print(f"{'n':<8} {'q':<10} {'α':<10} {'γ (approx.)':<15} {'Security'}")
print("-" * 60)

for i in range(len(n_values)):
    n = n_values[i]
    q = q_values[i]
    alpha = alpha_values[i]
    
    # Approximation factor: γ ≈ n/α (simplified)
    gamma = n / alpha
    
    # Very rough security estimate
    bits = n / 2
    
    print(f"{n:<8} {q:<10} {alpha:<10.4f} {gamma:<15.0f} ~{bits:.0f}-bit")

print("\nThese parameters give post-quantum security!")
print("Even quantum computers can't break worst-case SVP efficiently.")

## Quick Note on Error Distributions

In practice, errors in LWE aren't just random small integers - they're sampled from a **discrete Gaussian distribution** $D_{\mathbb{Z}^m, \sigma}$.

This gives probability $\propto \exp(-\pi \|x\|^2 / \sigma^2)$ to each point.

Picking $\sigma$ is a balancing act:
- Too small → LWE becomes easier to solve
- Too large → cryptographic schemes fail (decryption errors)

Typical values: $\sigma \approx 3.2$ for $q \approx 2^{15}$.

In [ ]:
# Comparing uniform vs Gaussian errors
np.random.seed(42)
m = 1000
sigma = 3.2

# Uniform errors from {-2, -1, 0, 1, 2}
uniform_errors = np.random.choice([-2, -1, 0, 1, 2], size=m)

# Discrete Gaussian (approximated)
gaussian_errors = np.round(np.random.normal(0, sigma, size=m))

print("Error Distribution Comparison:")
print("=" * 50)
print(f"\nUniform errors from {{-2,-1,0,1,2}}:")
print(f"Mean: {np.mean(uniform_errors):.3f}")
print(f"Std: {np.std(uniform_errors):.3f}")
print(f"Max |e|: {np.max(np.abs(uniform_errors))}")

print(f"\nGaussian errors with σ={sigma}:")
print(f"Mean: {np.mean(gaussian_errors):.3f}")
print(f"Std: {np.std(gaussian_errors):.3f}")
print(f"Max |e|: {np.max(np.abs(gaussian_errors)):.0f}")

# Norm of error vectors
norm_uniform = np.linalg.norm(uniform_errors[:10])
norm_gaussian = np.linalg.norm(gaussian_errors[:10])

print(f"\nNorm of 10-dimensional error vector:")
print(f"Uniform: {norm_uniform:.2f}")
print(f"Gaussian: {norm_gaussian:.2f}")
print(f"\nGaussian gives more predictable error sizes!")

## Summary: SIS is a Lattice Problem

Let's wrap this up! Why is SIS a lattice problem?

**1. It's literally about finding short lattice vectors**

SIS asks for a short $z$ with $Az = 0 \pmod{q}$. The set of all solutions is the lattice $L_A^\perp$. So SIS = find short vector in $L_A^\perp$ = approximate-SVP!

**2. It has worst-case hardness guarantees**

Thanks to Ajtai and Micciancio-Regev, we know:
- Breaking average-case SIS → Breaking worst-case SVP
- Approximation factor $\gamma = O(\sqrt{n})$ is sufficient
- This makes SIS provably hard, not just "probably hard"

That's why we trust SIS for cryptography!

In [14]:
# Visual summary: SIS lattice structure
n, m, q = 5, 15, 97

print("SIS Lattice Summary")
print("=" * 60)
print(f"\nParameters: n={n}, m={m}, q={q}")
print(f"\nLattice: L_A^perp = {{z ∈ Z^{m} : Az = 0 (mod {q})}}")
print(f"Dimension: {m} (full rank)")
print(f"Volume: q^n = {q**n:.2e}")

# Estimate lambda_1
lambda_1_est = np.sqrt(m / (2 * np.pi * np.e)) * (q ** (n/m))
print(f"\nEstimated λ₁: {lambda_1_est:.2f}")

# What beta makes SIS hard?
gamma_values = [1.0, 1.5, 2.0, 3.0]
print(f"\nApproximate-SVP hardness:")
for gamma in gamma_values:
    beta = gamma * lambda_1_est
    print(f"  γ={gamma}: need to find vector with ||z|| ≤ {beta:.1f}")

print(f"\nProblem: Given random A, find such a z")
print(f"Security: Based on worst-case SVP with γ = O(√{n}) ≈ {np.sqrt(n):.1f}")
print(f"\n✓ Post-quantum secure!")

SIS Lattice Summary

Parameters: n=5, m=15, q=97

Lattice: L_A^perp = {z ∈ Z^15 : Az = 0 (mod 97)}
Dimension: 15 (full rank)
Volume: q^n = 8.59e+09

Estimated λ₁: 4.31

Approximate-SVP hardness:
  γ=1.0: need to find vector with ||z|| ≤ 4.3
  γ=1.5: need to find vector with ||z|| ≤ 6.5
  γ=2.0: need to find vector with ||z|| ≤ 8.6
  γ=3.0: need to find vector with ||z|| ≤ 12.9

Problem: Given random A, find such a z
Security: Based on worst-case SVP with γ = O(√5) ≈ 2.2

✓ Post-quantum secure!


## Summary: LWE is a Lattice Problem

And why is LWE a lattice problem?

**1. It's bounded distance decoding**

LWE gives you $b = As + e$ where $e$ is small. The lattice $L_A$ contains $As$, and $b$ is nearby. Finding $As$ = solving BDD = lattice problem!

**2. BDD reduces to SVP**

Kannan's embedding shows BDD → SVP. So any SVP solver can crack LWE (if parameters are weak).

**3. Worst-case hardness (Regev's magic)**

Regev proved:
- Breaking average-case LWE → Breaking worst-case GapSVP/SIVP
- Uses quantum reduction, but classical versions exist
- Approximation factor $\gamma = \tilde{O}(n/\alpha)$

This is why LWE is the foundation of modern lattice crypto!

In [ ]:
# Visual summary: LWE lattice structure
n, m, q = 10, 30, 1009
alpha = 0.005
sigma = alpha * q

print("LWE Lattice Summary")
print("=" * 60)
print(f"\nParameters: n={n}, m={m}, q={q}")
print(f"Error rate: α={alpha}, σ={sigma:.2f}")
print(f"\nLattice: L_A = {{Az (mod {q}) : z ∈ Z^{n}}}")
print(f"Dimension: {n} (embedded in {m}D space)")
print(f"Volume: q^(m-n) = {q**(m-n):.2e}")

# Error size vs lattice spacing
error_norm = sigma * np.sqrt(m)
lambda_1_est = q ** ((m-n)/n)  # Rough estimate

print(f"\nTypical error ||e||: ~{error_norm:.2f}")
print(f"Estimated λ₁(L_A): ~{lambda_1_est:.2f}")
print(f"Ratio: ||e||/λ₁ ≈ {error_norm/lambda_1_est:.4f}")

if error_norm < lambda_1_est / 2:
    print("✓ Error is small enough - BDD has unique solution")
else:
    print("✗ Error too large - might have multiple close points")

# Hardness
gamma = n / alpha
print(f"\nSecurity: Based on GapSVP/SIVP with γ ≈ {gamma:.0f}")
print(f"Required approximation factor grows with n/α")
print(f"\n✓ Quantum-resistant security!")